In [1]:
from turtle import color
from xmlrpc.server import list_public_methods

from matplotlib import cm
import blackboxzig
import obj_fun
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import qmc
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import Axes3D
from scipy.stats import multivariate_normal
##---------------------------------------------
import torch
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_model
from botorch.utils import standardize
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.optim import optimize_acqf
from math import dist
from scipy.interpolate import griddata
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


ModuleNotFoundError: No module named 'microwaveopt'

In [21]:
#create initial data (10 pairs of parameters)
init_sample_num =10
input_dim = 2
bounds = np.array([[5,25],[0.3,1.2]])
sampler = qmc.LatinHypercube(d=input_dim)
init_points_normalized = sampler.random(n=init_sample_num)
init_points = (init_points_normalized * (bounds[:,1] - bounds[:,0]).T) + bounds[:,0].T
print(init_points)


[[10.364556    1.19780221]
 [22.94037823  0.87362452]
 [24.93260149  0.54058461]
 [18.89700863  0.3369358 ]
 [ 7.30597583  0.78267664]
 [ 6.15310069  1.08765088]
 [14.50930841  0.63946693]
 [15.09679969  0.67687673]
 [19.59635376  0.97046848]
 [12.49543869  0.42215703]]


In [22]:
init_values =obj_fun.obj_fun(x=init_points, debug=False)
#Save initial samples
init_samples = np.hstack((init_points, init_values))

------ RUNNING MOMENTUM SIMULATION ------
------ COMPLETED ------
------ RUNNING MOMENTUM SIMULATION ------
------ COMPLETED ------
------ RUNNING MOMENTUM SIMULATION ------
------ COMPLETED ------
------ RUNNING MOMENTUM SIMULATION ------
------ COMPLETED ------
------ RUNNING MOMENTUM SIMULATION ------
------ COMPLETED ------
------ RUNNING MOMENTUM SIMULATION ------
------ COMPLETED ------
------ RUNNING MOMENTUM SIMULATION ------
------ COMPLETED ------
------ RUNNING MOMENTUM SIMULATION ------
------ COMPLETED ------
------ RUNNING MOMENTUM SIMULATION ------
------ COMPLETED ------
------ RUNNING MOMENTUM SIMULATION ------
------ COMPLETED ------


In [23]:
train_X = torch.Tensor(init_samples[:,:-1])
train_Y = torch.Tensor(init_samples[:,-1].reshape(-1,1))

In [24]:
print(train_X.shape)
print(train_Y.shape)
iteretion =10;
ph=[25,1.2]
pl=[5,.3]
ps=[18.678,0.79]
regret=[]
counter=[]
for i in range(iteretion):

    count =i

    train_Y_norm = (train_Y - train_Y.mean()) / train_Y.std() # normalize values
    gp = SingleTaskGP(train_X, train_Y_norm)
    mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
    fit_gpytorch_model(mll)


# Compute acquisition function
    from botorch.acquisition import UpperConfidenceBound
    UCB = UpperConfidenceBound(gp, beta=0.1)

    # Maximize acquisition function
    acq_bounds = torch.Tensor(bounds.T)  # convert input bounds for the acquisition function
    next_point, acq_value = optimize_acqf(UCB, bounds=acq_bounds, q=1, num_restarts=5, raw_samples=20,)

    # Compute objective values at the next candidate point
    next_value = obj_fun.obj_fun(next_point, debug=False)
    next_value = torch.Tensor(next_value)

    # Add next sample to the dataset
    train_X = torch.vstack((train_X, next_point))
    train_Y = torch.vstack((train_Y, next_value))
    print (count)

    #Regret
    y_opt_idx = np.argmax(train_Y.numpy())
    y_opt= np.max(train_Y.numpy())
    x_opt = train_X.numpy()[y_opt_idx]
    regret.append(dist(x_opt,ps)/dist(ph,pl))
    counter.append(count)

plt.figure()
plt.plot(counter,regret)
plt.savefig(f".\\figures\\Regret1.png")

torch.Size([10, 2])
torch.Size([10, 1])
------ RUNNING MOMENTUM SIMULATION ------
------ COMPLETED ------
0
------ RUNNING MOMENTUM SIMULATION ------
------ COMPLETED ------
1
------ RUNNING MOMENTUM SIMULATION ------
------ COMPLETED ------
2
------ RUNNING MOMENTUM SIMULATION ------
------ COMPLETED ------
3
------ RUNNING MOMENTUM SIMULATION ------
------ COMPLETED ------
4
------ RUNNING MOMENTUM SIMULATION ------
------ COMPLETED ------
5
------ RUNNING MOMENTUM SIMULATION ------
------ COMPLETED ------
6
------ RUNNING MOMENTUM SIMULATION ------
------ COMPLETED ------
7
------ RUNNING MOMENTUM SIMULATION ------
------ COMPLETED ------
8
------ RUNNING MOMENTUM SIMULATION ------
------ COMPLETED ------
9


In [17]:
y_opt_idx = np.argmax(train_Y.numpy())
y_opt= np.max(train_Y.numpy())
x_opt = train_X.numpy()[y_opt_idx]
f ,s21 = blackboxzig.blackbox(x_opt, debug=True, simm='linear',fmin = 1 , fmax = 4)
plt.plot(f, 20*np.log10(np.abs(s21)))
plt.savefig(f".\\figures\\BestSolution.png")
with open ("BestSolution.txt",'a') as O:
            O.write(f"Best solution found: x={x_opt}, y={y_opt}")

------ RUNNING MOMENTUM SIMULATION ------
------ COMPLETED ------


In [25]:
#prediction
#create new inputs
init_sample_num =4000
input_dim = 2

bounds = np.array([[5,25],[0.3,1.2]])
sampler = qmc.LatinHypercube(d=input_dim)
init_points_normalized = sampler.random(n=init_sample_num)
init_points = (init_points_normalized * (bounds[:,1] - bounds[:,0]).T) + bounds[:,0].T
test_X = torch.Tensor(init_points)

gp.eval()
posterior = gp.posterior(test_X)
a=posterior.mean.cpu()
a = a.detach().numpy()

L_grid = np.linspace(5,25,200)
G_grid = np.linspace(.3,1.2,200)
L_mesh,G_mesh = np.meshgrid(L_grid,G_grid)


z_grid = griddata(test_X,a.flatten(),(L_mesh,G_mesh), method = 'linear')

%matplotlib qt 
fig = plt.figure()
ax =plt.axes(projection='3d', computed_zorder=False)
ax.plot_surface(L_mesh,G_mesh,z_grid,cmap=cm.coolwarm, alpha=0.8,zorder=-1)
ax.set_xlabel('L(mm)')
ax.set_ylabel('G(mm)')
ax.set_zlabel('E[q]')
ax.set_zlim3d(-10,0)
ax.scatter( train_X[:,0], train_X[:,1], train_Y,color='k')

plt.savefig(f".\\figures\\fig4.png")